In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 5
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 600

puzzle.naming = {
    None: ' ',
    'e': '_',
}
puzzle.latex_naming = {
    None: ' ',
    'e': ' ',
    '2': '\\Large \\ensuremath{2}',
    '3': '\\Large \\ensuremath{3}',
    '4': '\\Large \\ensuremath{4}',
    'num_nums': '{\\large \#}',
}

puzzle.domain_program += """
    value(e).
    num(2..4).
    value(N) :- num(N).
    
    1 { guessed_number(num_nums,4..14) } 1.
    max_id(N) :- guessed_number(num_nums,N).
    id(N) :- max_id(N).
    id(N-1) :- id(N), N > 1.
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : solution(C,V) }.
    :- puzzle(C,e).
"""
puzzle.puzzle_constraints_program += """
    #maximize { 1,solution(C,V) : solution(C,V), num(V), not puzzle(C,V) }.
    :- not solution(C,V) : cell(C), num(V).
    :- not #count { puzzle(C,V) : puzzle(C,V) } >= 3.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['cell_distance']
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += """
    solution(C) :- solution(C,V), num(V).
    first_solution(C1) :- solution(C1), C1 <= C2 : solution(C2).

    1 { map(C,I) : id(I) } 1 :- solution(C).
    1 { map(C,I) : solution(C) } 1 :- id(I).
    :- not map(C,1), first_solution(C).

    next_id(I,I+1) :- id(I), id(I+1).
    next_id(I,1) :- max_id(I).
    :- map(C1,I), map(C2,J), next_id(I,J), solution(C1,D), not cell_distance(C1,C2,D).
    
    :- adjacent_cells(C1,C2),
        solution(C1,D), num(D),
        solution(C2,E), num(E).
    
    :- guessed_number(num_nums,N),
        not N { solution(C,V) : solution(C,V), num(V) } N.
"""
puzzle.essential_solution_constraints = [
    """
    forbidden_cell_triple(C1,C2,C3) :-
        cell(C1), cell(C2), cell(C3),
        C1 = c(Row1,Col1), C2 = c(Row2,Col2), C3 = c(Row3,Col3),
        Row1 < Row2, Row2 < Row3,
        Col1 < Col2, Col2 < Col3.
    forbidden_cell_triple(C1,C2,C3) :-
        cell(C1), cell(C2), cell(C3),
        C1 = c(Row1,Col1), C2 = c(Row2,Col2), C3 = c(Row3,Col3),
        Row1 < Row2, Row2 < Row3,
        Col1 > Col2, Col2 > Col3.
    forbidden_cell_triple(C1,C2,C3) :-
        cell(C1), cell(C2), cell(C3),
        C1 = c(Row,Col1), C2 = c(Row,Col2), C3 = c(Row,Col3),
        Col1 < Col2, Col2 < Col3.
    forbidden_cell_triple(C1,C2,C3) :-
        cell(C1), cell(C2), cell(C3),
        C1 = c(Row1,Col), C2 = c(Row2,Col), C3 = c(Row3,Col),
        Row1 < Row2, Row2 < Row3.
    forbidden_cell_triple(C1,C2,C3) :- forbidden_cell_triple(C3,C2,C1).
    :- next_id(I1,I2), next_id(I2,I3),
        map(C1,I1), map(C2,I2), map(C3,I3),
        forbidden_cell_triple(C1,C2,C3).
    """,
    """
    forbidden_pair(D,E) :- num(D), num(E), |D-E| > 1.
    :- map(C1,I), map(C2,J), next_id(I,J),
        solution(C1,D), solution(C2,E), forbidden_pair(D,E).
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
    enforce_essential_constraints=True,
)

Done grounding..
...
Stopped after solving timeout..
Solving time: 2328.66 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| | | | |2| |
|4| |4| | | |
| | | | | | |
| | |3| | | |
| | | | |2| |

num_nums = 11


In [5]:
print(puzzle.pretty_repr_solution())

|_|3|_|_|2|_|
|4|_|4|_|_|3|
|_|2|_|_|3|_|
|_|_|3|_|_|2|
|_|4|_|_|2|_|


In [6]:
print(puzzle.latex_repr_puzzle())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) { };
      \node[anchor=center] (1;2) at (1.5,4.5) { };
      \node[anchor=center] (1;3) at (2.5,4.5) { };
      \node[anchor=center] (1;4) at (3.5,4.5) { };
      \node[anchor=center] (1;5) at (4.5,4.5) {\Large \ensuremath{2}};
      \node[anchor=center] (1;6) at (5.5,4.5) { };
      \node[anchor=center] (2;1) at (0.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] (2;2) at (1.5,3.5) { };
      \node[anchor=center] (2;3) at (2.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] (2;4) at (3.5,3.5) { };
      \node[anchor=center] (2;5) at (4.5,3.5) { };
      \node[anchor=center] (2;6) at (5.5,3.5) { };
      \node[anchor=center] (3;1) at (0.5,2.5) { };
      \node[anchor=center] (3;2) at (1.5,2.5) { };
      \node[anchor=center] (3;3) at (2.5,2.5) { };
      \node[anchor=center] (3;4) at (3.5,2.5) { }

In [7]:
print(puzzle.latex_repr_solution())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) { };
      \node[anchor=center] (1;2) at (1.5,4.5) {\Large \ensuremath{3}};
      \node[anchor=center] (1;3) at (2.5,4.5) { };
      \node[anchor=center] (1;4) at (3.5,4.5) { };
      \node[anchor=center] (1;5) at (4.5,4.5) {\Large \ensuremath{2}};
      \node[anchor=center] (1;6) at (5.5,4.5) { };
      \node[anchor=center] (2;1) at (0.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] (2;2) at (1.5,3.5) { };
      \node[anchor=center] (2;3) at (2.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] (2;4) at (3.5,3.5) { };
      \node[anchor=center] (2;5) at (4.5,3.5) { };
      \node[anchor=center] (2;6) at (5.5,3.5) {\Large \ensuremath{3}};
      \node[anchor=center] (3;1) at (0.5,2.5) { };
      \node[anchor=center] (3;2) at (1.5,2.5) {\Large \ensuremath{2}};
      \node[anchor=center] (3;3) at (2.5